In [3]:
import torch; 
import torch.nn as nn

### Using `__setattr__`

- the DummyModule class code below is analogous to what PyTorch nn.Module does behind the scene that automatically registers sub-modules defined under `__init__` as parameters of the model object

In [80]:
class DummyModule():
    def __init__(self, n_in, nh, n_out):
        self._modules = {}
        self.l1 = nn.Linear(n_in,nh)   # sub-module, get registered under self._modules
        self.l2 = nn.Linear(nh,n_out)  # sub-module, get registered under self._modules
         
    def __setattr__(self, k, v):
        if not k.startswith("_"): self._modules[k] = v
        super().__setattr__(k,v)
        
    def __repr__(self): return f'{self._modules}' 
    
    def parameters(self):
        for l in self._modules.values():
            for p in l.parameters(): yield p

In [81]:
dummy = DummyModule(512, hl, n_out)
dummy._modules

{'l1': Linear(in_features=512, out_features=3, bias=True),
 'l2': Linear(in_features=3, out_features=10, bias=True)}

In [96]:
dummy

{'l1': Linear(in_features=512, out_features=3, bias=True), 'l2': Linear(in_features=3, out_features=10, bias=True)}

In [89]:
# note that the __repr__ method defines the string representation of the object
assert dummy.__repr__() == str(dummy)

In [95]:
[o.shape for o in dummy.parameters()]

[torch.Size([3, 512]), torch.Size([3]), torch.Size([10, 3]), torch.Size([10])]

### Simple model

- Below is a simple fully connected neural net with only a single hiddeln layer, followed by ReLU activation and output layer
- Note the usage of `__call__` in Python allows the class object to be called like a function method

In [111]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.layers = layers
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [112]:
bs, hl, n_out = 64, 3, 10
x = torch.randn(bs, 512)  # create random input of batch
m = Model(512, hl, n_out)
m

Model()

### nn.Module

- Note that the list of layers we defined did not get registered under the model object propperty
- The code below registers of the list of modules manually by calling the `nn.Module.add_module` method

In [113]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.layers = layers
        for i,l in enumerate(self.layers): self.add_module(f'layer_{i}', l)
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [114]:
bs, hl, n_out = 64, 3, 10
x = torch.randn(bs, 512)  # create random input of batch
m = Model(512, hl, n_out)
m

Model(
  (layer_0): Linear(in_features=512, out_features=3, bias=True)
  (layer_1): ReLU()
  (layer_2): Linear(in_features=3, out_features=10, bias=True)
)

### nn.ModuleList
- conveniently PyTorch does this automatically for us via the `nn.ModuleList` method

In [115]:
class SequentialModel(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [116]:
layers = [nn.Linear(512,3), nn.ReLU(), nn.Linear(3,10)]

In [119]:
m = SequentialModel(layers); m

SequentialModel(
  (layers): ModuleList(
    (0): Linear(in_features=512, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=10, bias=True)
  )
)

### nn.Sequential

- alternatively, the `nn.Sequential` method does the same thing

In [121]:
m = nn.Sequential(nn.Linear(512,3), nn.ReLU(), nn.Linear(3,10)); m

Sequential(
  (0): Linear(in_features=512, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=10, bias=True)
)